In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

sys.path.append('/coc/scratch/anarayanan68/mint/')

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, initializers, regularizers

from mint.core import fact_model, base_model_util, base_models, primitive_models, model_builder
from mint.utils import inputs_util, config_util

In [4]:
config_path = '/coc/scratch/anarayanan68/mint/configs/audio_based_blending__10clips10prims.config'
enc_config_path = '/coc/scratch/anarayanan68/mint/configs/audio_based_blending__10clips100prims-enc_config.yml'

In [5]:
expt_root = '/coc/scratch/anarayanan68/mint/_expts/audio_based_blending__nonconst_embed_init'

checkpoint_dir = os.path.join(expt_root, 'checkpoints/')

In [6]:
configs = config_util.get_configs_from_pipeline_file(config_path)
model_config = configs['model']
eval_config = configs['eval_config']
eval_dataset_config = configs['eval_dataset']

In [7]:
enc_config_yaml = config_util.read_yaml_config(enc_config_path)

In [8]:
# Model build & restore

model = model_builder.build(model_config, is_training=False,
    encoder_config_yaml=enc_config_yaml, dataset_config=eval_dataset_config)

checkpoint_manager=tf.train.CheckpointManager(
    tf.train.Checkpoint(model=model),
    directory=checkpoint_dir,
    max_to_keep=None)

checkpoint_manager.restore_or_initialize()

'/coc/scratch/anarayanan68/mint/_expts/audio_based_blending__nonconst_embed_init/checkpoints/ckpt-999'

In [10]:
num_primitives = enc_config_yaml['num_primitives']
all_primitives = model.blend_vec_to_seq(np.eye(num_primitives))
all_primitives

<tf.Tensor: shape=(100, 120, 147), dtype=float32, numpy=
array([[[ 5.42774843e-03, -1.79649685e-02,  7.54127884e-03, ...,
         -1.76014996e-03,  6.09469041e-03, -2.54331203e-03],
        [ 9.18337144e-03,  2.19735070e-04, -1.01251705e-02, ...,
          2.44990736e-03,  8.71144701e-03,  1.80374142e-02],
        [ 8.53181258e-03,  1.35866981e-02,  1.11544728e-02, ...,
          1.83612145e-02,  3.67516768e-05, -6.54187379e-03],
        ...,
        [-3.26880417e-03, -1.56684965e-02, -7.22290203e-03, ...,
          1.80533826e-02,  1.36567531e-02,  6.36316184e-03],
        [-1.20885717e-02, -5.07602282e-03,  1.15867646e-03, ...,
          1.59240160e-02, -1.46949599e-02,  1.22828381e-02],
        [ 1.68150309e-02,  1.25663029e-02, -8.08002800e-03, ...,
          9.32927895e-03, -2.76051369e-03, -1.40346056e-02]],

       [[-6.08389312e-03,  1.32947313e-02, -7.45249446e-03, ...,
         -1.17176389e-02,  1.24196829e-02,  5.98192960e-03],
        [-1.00439386e-02, -4.44061914e-03, -1.

In [11]:
save_dir = os.path.join(expt_root, 'primitives')
os.makedirs(save_dir, exist_ok=True)
for i in range(num_primitives):
    np.save(os.path.join(save_dir, f"PRIMITIVE--{i+1}_of_{num_primitives}.npy"), all_primitives[i])